In [1]:
import sklearn
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

In [2]:
tf.constant([[1., 2., 3.], [4., 5., 6.]]) # matrix

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
tf.constant(42) # scalar

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [4]:
t = tf.constant([[1., 2., 3.], [4., 5., 6]])
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [5]:
t.shape

TensorShape([2, 3])

In [6]:
t.dtype

tf.float32

In [7]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [8]:
a = np.array([1., 4., 5.])

In [9]:
a

array([1., 4., 5.])

In [10]:
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([1., 4., 5.])>

In [11]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 1., 16., 25.])>

In [12]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [13]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

In [14]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2] name: add/

In [15]:
tf.constant(2.) + tf.constant(4., dtype=tf.float64)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2] name: add/

In [16]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

In [20]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])

In [21]:
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [22]:
v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [23]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [25]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [40]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [30]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [31]:
housing = fetch_california_housing()

In [32]:
x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target.reshape(-1, 1), random_state=42)

In [34]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, random_state=42)

In [35]:
scaler = StandardScaler()

In [36]:
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [37]:
input_shape = x_train.shape[1:]

In [38]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='selu', kernel_initializer='lecun_normal', input_shape=input_shape),
    keras.layers.Dense(1)
])

In [41]:
model.compile(loss=huber_fn, metrics=['mae'], optimizer='nadam')

In [42]:
history = model.fit(x_train_scaled, y_train, epochs=2, validation_data=[x_valid_scaled, y_valid])

Train on 11610 samples, validate on 3870 samples
Epoch 1/2
11610/11610 [==============================] - 1s 85us/sample - loss: 0.5394 - mae: 0.8897 - val_loss: 0.2128 - val_mae: 0.5033
Epoch 2/2
11610/11610 [==============================] - 0s 34us/sample - loss: 0.2077 - mae: 0.4997 - val_loss: 0.2080 - val_mae: 0.4916


In [43]:
model.save('model_with_custom_loss.h5')

In [44]:
model = keras.models.load_model('model_with_custom_loss.h5', custom_objects={'huber_fn': huber_fn})

In [45]:
model.fit(x_train_scaled, y_train, epochs=2, validation_data=(x_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/2
11610/11610 [==============================] - 1s 61us/sample - loss: 0.2010 - mae: 0.4900 - val_loss: 0.1927 - val_mae: 0.4739
Epoch 2/2
11610/11610 [==============================] - 0s 37us/sample - loss: 0.1976 - mae: 0.4853 - val_loss: 0.1824 - val_mae: 0.4605


In [48]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [52]:
model.compile(loss=create_huber(2.0), optimizer='nadam', metrics=['mae'])

In [54]:
history = model.fit(x_train_scaled, y_train, epochs=2, validation_data=(x_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/2
11610/11610 [==============================] - 1s 69us/sample - loss: 0.2204 - mae: 0.4862 - val_loss: 0.2063 - val_mae: 0.4657
Epoch 2/2
11610/11610 [==============================] - 0s 38us/sample - loss: 0.2171 - mae: 0.4834 - val_loss: 0.2160 - val_mae: 0.4685


In [55]:
model.save('model_with_create_loss.h5')

In [58]:
model = keras.models.load_model('model_with_create_loss.h5', custom_objects={'huber_fn': create_huber(2.0)})